In [1]:
!pip install pystan
!pip install fbprophet

In [2]:
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')
from fbprophet import Prophet

In [3]:
df = pd.read_excel('Winer Olympic Medals.xlsx', sheet_name='Data')
df.head()

,Year,Sport,Event,Country,Gender,Medal Rank,Medal,Name of Athlete or Team,Age of Athlete
0,1924,Bobsled,Men's Four/Five,Switzerland,Men,1,gold,Switzerland-1,NaN
1,1924,Bobsled,Men's Four/Five,Britain,Men,2,silver,Britain-1,NaN
2,1924,Bobsled,Men's Four/Five,Belgium,Men,3,bronze,Belgium-1,NaN
3,1924,Cross-Country Skiing,Men's 18 Kilometers,Norway,Men,1,gold,Thorleif Haug,29.0
4,1924,Cross-Country Skiing,Men's 18 Kilometers,Norway,Men,2,silver,Johan GrÃ¸ttumsbraaten,24.0


In [4]:
df.drop(['Event','Age of Athlete', 'Gender', 'Medal Rank', 'Name of Athlete or Team'], axis = 1, inplace=True)

In [5]:
df['Year'].nunique()

22

In [6]:
df['Country'].unique()

array(['Switzerland', 'Britain', 'Belgium', 'Norway', 'Finland', 'Sweden',
       'France', 'Austria', 'United States', 'Canada', 'Germany',
       'Czechoslovakia', 'Hungary', 'Italy', 'Netherlands', 'Japan',
       'Soviet Union', 'Poland', 'North Korea', 'West Germany', 'Romania',
       'East Germany', 'Spain', 'Liechtenstein', 'Bulgaria', 'Yugoslavia',
       'Luxembourg', 'New Zealand', 'Unified Team', 'South Korea',
       'China', 'Slovenia', 'Russia', 'Belarus', 'Ukraine', 'Kazakhstan',
       'Uzbekistan', 'Australia', 'Czech Republic', 'Denmark', 'Croatia',
       'Estonia', 'Latvia', 'Slovakia'], dtype=object)

In [7]:
countries = list(df['Country'].unique())

In [8]:
df.head()

,Year,Sport,Country,Medal
0,1924,Bobsled,Switzerland,gold
1,1924,Bobsled,Britain,silver
2,1924,Bobsled,Belgium,bronze
3,1924,Cross-Country Skiing,Norway,gold
4,1924,Cross-Country Skiing,Norway,silver


In [9]:
df_t = pd.get_dummies(data=df, columns=['Medal'])
df_t.head()

,Year,Sport,Country,Medal_bronze,Medal_gold,Medal_silver
0,1924,Bobsled,Switzerland,0,1,0
1,1924,Bobsled,Britain,0,0,1
2,1924,Bobsled,Belgium,1,0,0
3,1924,Cross-Country Skiing,Norway,0,1,0
4,1924,Cross-Country Skiing,Norway,0,0,1


In [10]:
df_t['Year'] = pd.to_datetime(df_t['Year'], format='%Y')

In [11]:
df_t.rename({'Year':'ds'}, inplace=True, axis=1)

In [12]:
df_t.head()

,ds,Sport,Country,Medal_bronze,Medal_gold,Medal_silver
0,1924-01-01,Bobsled,Switzerland,0,1,0
1,1924-01-01,Bobsled,Britain,0,0,1
2,1924-01-01,Bobsled,Belgium,1,0,0
3,1924-01-01,Cross-Country Skiing,Norway,0,1,0
4,1924-01-01,Cross-Country Skiing,Norway,0,0,1


In [13]:
country_medals = df_t.groupby(['Country', 'ds'])[['Medal_bronze', 'Medal_gold', 'Medal_silver']].sum()
country_medals.head()

Medal_bronze  Medal_gold  Medal_silver
Country   ds                                                
Australia 1994-01-01             1           0             0
          1998-01-01             1           0             0
          2002-01-01             0           2             0
          2006-01-01             1           1             0
          2010-01-01             0           2             1

In [14]:
country_medals.loc['Romania']

,Medal_bronze,Medal_gold,Medal_silver
ds,,,
1968-01-01,1,0,0


In [15]:
from fbprophet.serialize import model_to_json

for country in countries:
    target_df = country_medals.loc[country].reset_index()
    if target_df.shape[0] < 2:
        continue

    bronze = target_df[['ds', 'Medal_bronze']]
    bronze.rename({'Medal_bronze':'y'}, inplace=True, axis=1)

    silver = target_df[['ds', 'Medal_silver']]
    silver.rename({'Medal_silver':'y'}, inplace=True, axis=1)

    gold = target_df[['ds', 'Medal_gold']]
    gold.rename({'Medal_gold':'y'}, inplace=True, axis=1)

    m_bronze, m_silver, m_gold = Prophet(), Prophet(), Prophet()
    print(country)
    #print(bronze.head())
    try:
        m_bronze.fit(bronze)
    except ValueError as e:
        raise Exception("Bad df")

    try:
        m_silver.fit(silver)
    except m_bronze as e:
        print(m_bronze)
        raise Exception("Bad df")
    
    try:
        m_gold.fit(gold)
    except ValueError as e:
        print(m_bronze)
        raise Exception("Bad df")

    with open(f'{country}_bronze.json', 'w') as fout:
        json.dump(model_to_json(m_bronze), fout)

    with open(f'{country}_silver.json', 'w') as fout:
        json.dump(model_to_json(m_silver), fout)

    with open(f'{country}_gold.json', 'w') as fout:
        json.dump(model_to_json(m_gold), fout)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.


Switzerland


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 11.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

Britain


INFO:fbprophet:n_changepoints greater than number of observations. Using 11.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 11.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 2.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 2.


...

Estonia


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.


Latvia


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 1.


Slovakia


In [16]:
from fbprophet.serialize import model_to_json, model_from_json
with open('Australia_gold.json', 'r') as fin:
    m = model_from_json(json.load(fin))  # Load model

In [17]:
future = m.make_future_dataframe(periods=365*4)
future.tail()

,ds
1461,2017-12-27
1462,2017-12-28
1463,2017-12-29
1464,2017-12-30
1465,2017-12-31


In [18]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1461,2017-12-27,1.266739,0.254783,2.256355
1462,2017-12-28,1.289320,0.220393,2.360145
1463,2017-12-29,1.307372,0.341987,2.306748
1464,2017-12-30,1.320548,0.265812,2.377824
1465,2017-12-31,1.328595,0.261897,2.395833


In [22]:
forecast['yhat'].sum() / len(forecast['yhat'])

0.791320210325088

In [23]:
int(round(forecast['yhat'].sum() / len(forecast['yhat'])))

1